# Measuring the Liquid Engine overhead

### Lets build a Liquid Engine class where the runtypes are always known, via well-defined sleep calls

The runtimes are as follows:
1. OpenCL: user defined argument in seconds; 

2. Threaded: 3 seconds;

3. Unthreaded: 5 seconds; 

---

In [1]:
from timeit import default_timer
import numpy as np

from OverheadTester import OverheadTester
ot = OverheadTester(clear_benchmarks=True)
for _ in range(3):
    ot.benchmark(0.1)

Optional dependency Cupy is not installed. Cupy implementations will be ignored.
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10502112493850291 seconds
Agent: OverheadTester using Threaded ran in 0.5014944579452276 seconds
Agent: OverheadTester using Unthreaded ran in 1.0050573751796037 seconds
Fastest run type: OpenCL_Apple M1 Pro
Slowest run type: Unthreaded
OpenCL_Apple M1 Pro is 4.78x faster than Threaded
OpenCL_Apple M1 Pro is 9.57x faster than Unthreaded
Threaded is 2.00x faster than Unthreaded
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10577679192647338 seconds
Agent: OverheadTester using Threaded ran in 0.5050931249279529 seconds
Agent: OverheadTester using Unthreaded ran in 1.0039819171652198 seconds
Fastest run type: OpenCL_Apple M1 Pro
Slowest run type: Unthreaded
OpenCL_Apple M1 Pro is 4.78x faster than Threaded
OpenCL_Apple M1 Pro is 9.49x faster than Unthreaded
Threaded is 1.99x faster than Unthreaded
Agent: OverheadTester using OpenCL_Apple M1 P

---
### Let's run each implementation 100 times

In [2]:
def _opencl(_ot):
    t1 = default_timer()
    _ot.run(0.1,run_type="OpenCL_Apple M1 Pro")
    t2 = default_timer()
    return t2-t1

def _unth(_ot):
    t1 = default_timer()
    _ot.run(0.1,run_type="Unthreaded")
    t2 = default_timer()
    return t2-t1

def _thr(_ot):
    t1 = default_timer()
    _ot.run(0.1,run_type="Threaded")
    t2 = default_timer()
    return t2-t1

ocltimes = []
unttimes = []
thrtimes = []
for _ in range(100):
    ocltimes.append(_opencl(ot))
    unttimes.append(_unth(ot))
    thrtimes.append(_thr(ot))

Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10257091699168086 seconds
Agent: OverheadTester using Unthreaded ran in 1.0061008750926703 seconds
Agent: OverheadTester using Threaded ran in 0.5050538328941911 seconds
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10505466698668897 seconds
Agent: OverheadTester using Unthreaded ran in 1.0050541670061648 seconds
Agent: OverheadTester using Threaded ran in 0.5050578750669956 seconds
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10138549981638789 seconds
Agent: OverheadTester using Unthreaded ran in 1.0050509159918875 seconds
Agent: OverheadTester using Threaded ran in 0.5034357088152319 seconds
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10503583308309317 seconds
Agent: OverheadTester using Unthreaded ran in 1.0058303747791797 seconds
Agent: OverheadTester using Threaded ran in 0.5056860412005335 seconds
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10506216692738235 seconds
Agent: Ov

--- 
### Let's run the method 100 times but letting the agent decide the method

In [3]:
def _liquid(_ot):
    t1 = default_timer()
    _ot.run(0.1)
    t2 = default_timer()
    return t2-t1

liquidtimes = []
for _ in range(100):
    liquidtimes.append(_liquid(ot))

Querying the Agent...
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10505554103292525 seconds
Querying the Agent...
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10122824995778501 seconds
Querying the Agent...
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10839862516149879 seconds
Querying the Agent...
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10508195916190743 seconds
Querying the Agent...
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.105071583064273 seconds
Querying the Agent...
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10493058315478265 seconds
Querying the Agent...
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10479274997487664 seconds
Querying the Agent...
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.10505791590549052 seconds
Querying the Agent...
Agent: OverheadTester using OpenCL_Apple M1 Pro ran in 0.1050425001885742 seconds
Querying the Agent...
Agent: OverheadTester using OpenCL_A

---
### Let's also measure the time taken for a simple python call. This will help to calculate the overhead of profiling and see if a pure python function is remarkably better than having a Liquid Engine class

In [4]:
from time import sleep

def _purepyth():
    t1 = default_timer()
    sleep(0.1)
    t2 = default_timer()
    return t2-t1

pythtimes = []
for _ in range(100):
    pythtimes.append(_purepyth())

--- 
---
## We are ready to measure the overheads.

#### Let's first measure the overhead of having a Liquid Engine class calling a method. This encompasses the time the Liquid Engine takes for record keeping (runtime I/O)

In [5]:
print(f"{np.average(ocltimes)-0.1:.4f} seconds")
print(f"{np.average(thrtimes)-0.5:.4f} seconds")
print(f"{np.average(unttimes)-1:.4f} seconds")

0.0100 seconds
0.0111 seconds
0.0155 seconds


#### Let's measure the overhead of the agent picking an implementation

In [6]:
print(f"{np.average(liquidtimes)-0.1:.4f} seconds")

0.0218 seconds


#### Finally, let's measure a simple python function call

In [7]:
print(f"{np.average(pythtimes)-0.1:.4f} seconds")

0.0037 seconds
